In [21]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
import os
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-20T03:30:00"
event_start = "2010-01-20"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/gridrad_demo_{event_start.replace('-', '')}"

In [10]:
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)
lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)
# for testing, ignore the era5 data
# data_options = option.consolidate_options([gridrad_options])

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)
gridrad_options = data_options["gridrad"]

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options = None

2024-10-21 17:49:06,226 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.


In [7]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [8]:
times = data.utils.generate_times(data_options["gridrad"])
args = [times, data_options.copy(), grid_options.copy()]
args += [track_options.model_copy(), visualize_options]
%memit track.simultaneous_track(*args, output_directory=output_parent)

2024-10-21 17:30:44,018 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo_20100121.
2024-10-21 17:30:44,024 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-21 17:30:44,083 - thor.track - INFO - Processing 2010-01-21T12:00:00.
2024-10-21 17:30:44,084 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T12:00:00.
2024-10-21 17:30:44,085 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100121T120000Z.nc


Filename: /home/ewan/Documents/THOR/thor/data/gridrad.py

Line #    Mem usage    Increment  Occurences   Line Contents
   117   1825.5 MiB   1825.5 MiB           1   @profile
   118                                         def reshape_variable(ds, variable):
   119                                             """
   120                                             Reshape a variable in a GridRad dataset to a 3D grid. Adapted from code provided by
   121                                             Stacey Hitchcock.
   122                                             """
   123                                         
   124   1836.3 MiB     10.9 MiB           1       values = ds[variable].values
   125   1836.3 MiB      0.0 MiB           1       attrs = ds[variable].attrs
   126   1836.3 MiB      0.0 MiB           1       alt, lat, lon = ds["Altitude"], ds["Latitude"], ds["Longitude"]
   127   1836.3 MiB      0.0 MiB           1       new_values = np.zeros(len(alt) * len(lat) * len(lon))
  

2024-10-21 17:30:57,493 - thor.track - INFO - Processing hierarchy level 0.
2024-10-21 17:30:57,494 - thor.track - INFO - Tracking convective.
2024-10-21 17:31:19,532 - thor.track - INFO - Tracking middle.
2024-10-21 17:31:25,278 - thor.track - INFO - Tracking anvil.
2024-10-21 17:31:27,757 - thor.track - INFO - Processing hierarchy level 1.
2024-10-21 17:31:27,758 - thor.track - INFO - Tracking mcs.
2024-10-21 17:31:39,836 - thor.match.match - INFO - Matching mcs objects.
2024-10-21 17:31:39,858 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-21 17:31:40,009 - thor.track - INFO - Processing 2010-01-21T12:10:00.
2024-10-21 17:31:40,011 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T12:10:00.
2024-10-21 17:31:40,012 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100121T121000Z.nc


Filename: /home/ewan/Documents/THOR/thor/data/gridrad.py

Line #    Mem usage    Increment  Occurences   Line Contents
   117   2418.9 MiB   2418.9 MiB           1   @profile
   118                                         def reshape_variable(ds, variable):
   119                                             """
   120                                             Reshape a variable in a GridRad dataset to a 3D grid. Adapted from code provided by
   121                                             Stacey Hitchcock.
   122                                             """
   123                                         
   124   2418.9 MiB      0.0 MiB           1       values = ds[variable].values
   125   2418.9 MiB      0.0 MiB           1       attrs = ds[variable].attrs
   126   2418.9 MiB      0.0 MiB           1       alt, lat, lon = ds["Altitude"], ds["Latitude"], ds["Longitude"]
   127   2418.9 MiB      0.0 MiB           1       new_values = np.zeros(len(alt) * len(lat) * len(lon))
  

2024-10-21 17:31:53,408 - thor.data.era5 - INFO - Updating era5_pl dataset for 2010-01-21T12:00:00.
2024-10-21 17:31:55,278 - thor.data.era5 - INFO - Updating era5_sl dataset for 2010-01-21T12:00:00.
2024-10-21 17:31:55,439 - thor.track - INFO - Processing hierarchy level 0.
2024-10-21 17:31:55,440 - thor.track - INFO - Tracking convective.
2024-10-21 17:32:17,086 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:32:17,089 - thor.track - INFO - Tracking middle.
2024-10-21 17:32:23,404 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:32:23,406 - thor.track - INFO - Tracking anvil.
2024-10-21 17:32:25,662 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:32:25,663 - thor.track - INFO - Processing hierarchy level 1.
2024-10-21 17:32:25,664 - thor.track - INFO - Tracking mcs.
2024-10-21 17:32:25,693 - thor.write.mask - INFO - Writing mcs masks to /home/ewan/THOR_output/runs/gridrad_demo_20100121/masks/m

...

Filename: /home/ewan/Documents/THOR/thor/data/gridrad.py

Line #    Mem usage    Increment  Occurences   Line Contents
   117   3796.2 MiB   3796.2 MiB           1   @profile
   118                                         def reshape_variable(ds, variable):
   119                                             """
   120                                             Reshape a variable in a GridRad dataset to a 3D grid. Adapted from code provided by
   121                                             Stacey Hitchcock.
   122                                             """
   123                                         
   124   3796.2 MiB      0.0 MiB           1       values = ds[variable].values
   125   3796.2 MiB      0.0 MiB           1       attrs = ds[variable].attrs
   126   3796.2 MiB      0.0 MiB           1       alt, lat, lon = ds["Altitude"], ds["Latitude"], ds["Longitude"]
   127   3796.2 MiB      0.0 MiB           1       new_values = np.zeros(len(alt) * len(lat) * len(lon))
  

2024-10-21 17:35:42,210 - thor.track - INFO - Processing hierarchy level 0.
2024-10-21 17:35:42,211 - thor.track - INFO - Tracking convective.
2024-10-21 17:36:03,381 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:36:03,382 - thor.track - INFO - Tracking middle.
2024-10-21 17:36:09,531 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:36:09,531 - thor.track - INFO - Tracking anvil.
2024-10-21 17:36:11,566 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:36:11,567 - thor.track - INFO - Processing hierarchy level 1.
2024-10-21 17:36:11,568 - thor.track - INFO - Tracking mcs.
2024-10-21 17:36:11,594 - thor.write.mask - INFO - Writing mcs masks to /home/ewan/THOR_output/runs/gridrad_demo_20100121/masks/mcs.zarr.
2024-10-21 17:36:23,631 - thor.match.match - INFO - Matching mcs objects.
2024-10-21 17:36:24,144 - thor.match.match - INFO - Updating object record.
2024-10-21 17:36:24,265 - thor.attribute.

Filename: /home/ewan/Documents/THOR/thor/data/gridrad.py

Line #    Mem usage    Increment  Occurences   Line Contents
   117   3752.6 MiB   3752.6 MiB           1   @profile
   118                                         def reshape_variable(ds, variable):
   119                                             """
   120                                             Reshape a variable in a GridRad dataset to a 3D grid. Adapted from code provided by
   121                                             Stacey Hitchcock.
   122                                             """
   123                                         
   124   3752.6 MiB      0.0 MiB           1       values = ds[variable].values
   125   3752.6 MiB      0.0 MiB           1       attrs = ds[variable].attrs
   126   3752.6 MiB      0.0 MiB           1       alt, lat, lon = ds["Altitude"], ds["Latitude"], ds["Longitude"]
   127   3752.6 MiB      0.0 MiB           1       new_values = np.zeros(len(alt) * len(lat) * len(lon))
  

2024-10-21 17:36:38,359 - thor.write.filepath - INFO - Writing gridrad filepaths from 2010-01-21T12 to 2010-01-21T13, inclusive and non-inclusive, respectively.
2024-10-21 17:36:38,372 - thor.track - INFO - Processing hierarchy level 0.
2024-10-21 17:36:38,375 - thor.track - INFO - Tracking convective.
2024-10-21 17:36:38,379 - thor.write.attribute - INFO - Writing convective attributes from 2010-01-21T12:00:00 to 2010-01-21T13:00:00, inclusive and non-inclusive, respectively.
2024-10-21 17:36:58,037 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:36:58,038 - thor.track - INFO - Tracking middle.
2024-10-21 17:36:58,041 - thor.write.attribute - INFO - Writing middle attributes from 2010-01-21T12:00:00 to 2010-01-21T13:00:00, inclusive and non-inclusive, respectively.
2024-10-21 17:37:03,745 - thor.attribute.attribute - INFO - Recording object attributes.
2024-10-21 17:37:03,745 - thor.track - INFO - Tracking anvil.
2024-10-21 17:37:03,747 - thor.write.attr

peak memory: 5885.63 MiB, increment: 4060.18 MiB


In [11]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 2

with log.logging_listener(), multiprocessing.get_context("spawn").Pool(
    initializer=parallel.initialize_process, processes=num_processes
) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.model_copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



2024-10-21 17:49:44,027 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo_20100121/interval_0.
2024-10-21 17:49:44,028 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-21 17:49:44,068 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo_20100121/interval_1.
2024-10-21 17:49:44,069 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-21 17:49:44,118 - thor.track - INFO - Processing 2010-01-21T12:00:00.
2024-10-21 17:49:44,118 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T12:00:00.
2024-10-21 17:49:44,119 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100121T120000Z.nc
2024-10-21 17:49:44,169 - thor.track - INFO - Processing 2010-01-21T13:00:00.
2024-10-21 17:49:44,170 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T13:00:00.
2024-10-21 17:49:44,170 - thor.data.gridrad - INFO - Converting gridrad d

In [12]:
parallel.stitch_run(output_parent, intervals, cleanup=False)

2024-10-21 17:59:13,866 - thor.parallel - INFO - Stitching all attribute, mask and record files.


2024-10-21 17:59:16,448 - thor.parallel - INFO - Stitching record files.
2024-10-21 17:59:16,477 - thor.parallel - INFO - Stitching attribute files.
2024-10-21 17:59:16,958 - thor.parallel - INFO - Stitching mask files.


In [17]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent)

convective_contained  anvil_contained  \
time                universal_id                                          
2010-01-21 12:00:00 1                             True             True   
2010-01-21 12:10:00 1                             True             True   
                    2                             True             True   
2010-01-21 12:20:00 1                             True             True   
                    3                             True             True   
2010-01-21 12:30:00 1                             True             True   
                    3                             True             True   
2010-01-21 12:40:00 1                             True             True   
                    3                             True             True   
2010-01-21 12:50:00 1                             True             True   
                    3                             True             True   
2010-01-21 13:00:00 1                             True             True   
                    3                             True             True   
2010-01-21 13:10:00 1                             True             True   
                    3                             True             True   
2010-01-21 13:20:00 1                             True             True   
                    3                             True             True   
2010-01-21 13:30:00 1                             True             True   
                    3                             True             True   
                    4                             True             True   
2010-01-21 13:40:00 1                             True             True   
                    3                             True             True   
2010-01-21 13:50:00 1                             True             True   
                    3                             True             True   
                    5                             True             True   

                                  velocity  area  offset  major_axis  \
time                universal_id                                       
2010-01-21 12:00:00 1                 True  True    True        True   
2010-01-21 12:10:00 1                 True  True    True        True   
                    2                 True  True    True        True   
2010-01-21 12:20:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 12:30:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 12:40:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 12:50:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 13:00:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 13:10:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 13:20:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 13:30:00 1                 True  True    True        True   
                    3                 True  True    True        True   
                    4                 True  True    True        True   
2010-01-21 13:40:00 1                 True  True    True        True   
                    3                 True  True    True        True   
2010-01-21 13:50:00 1                 True  True    True        True   
                    3                 True  True    True        True   
                    5                 True  True    True        True   

                                  axis_ratio  duration  
time  

In [11]:
%load_ext memory_profiler

In [23]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2010-01-20T22:00")
end_time = np.datetime64(np.datetime64("2010-01-20T23:00"))
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": False, "dt": 7200, "by_date": False, "num_processes": (os.cpu_count()-2)}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-22 14:15:23,340 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:00:00.000000000.
2024-10-22 14:15:27,860 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-22 14:15:35,152 - thor.visualize.attribute - INFO - Saving mcs_velocity_analysis figure for 2010-01-20T22:00:00.000000000.
2024-10-22 14:15:48,090 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:10:00.000000000.
2024-10-22 14:15:52,103 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-22 14:15:59,894 - thor.visualize.attribute - INFO - Saving mcs_velocity_analysis figure for 2010-01-20T22:10:00.000000000.
2024-10-22 14:16:14,618 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T22:20:00.000000000.
2024-10-22 14:16:18,752 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-22 14:16:25,972 - thor.visualize.attribute - INFO - Saving mcs_velocity_analysis figure for 20